<a href="https://colab.research.google.com/github/likhithaap/Self-correcting-coding-assistant/blob/main/codingAssistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# First install required packages
!pip install langsmith langchain-together

import os
from langchain import LLMChain
from langchain.prompts import PromptTemplate
from langchain_together import ChatTogether
from langsmith import Client
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks import LangChainTracer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.0/55.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.2/413.2 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.33
    Uninstalling langchain-core-0.3.33:
      Successfully uninstalled langchain-core-0.3.33


In [2]:

import logging

# Enable debug logging
logging.basicConfig(level=logging.DEBUG)

# Function to verify environment setup
def verify_environment():
    required_vars = ["LANGCHAIN_API_KEY", "LANGCHAIN_PROJECT", "TOGETHER_API_KEY"]
    missing_vars = [var for var in required_vars if not os.getenv(var)]
    if missing_vars:
        raise ValueError(f"Missing required environment variables: {missing_vars}")
    print("Environment variables verified ✓")

# Function to test LangSmith connection
def test_langsmith_connection():
    try:
        client = Client()
        # Try to list projects to verify connection
        client.list_projects()
        print("LangSmith connection successful ✓")
        return client
    except Exception as e:
        print(f"LangSmith connection failed: {str(e)}")
        raise


In [3]:


# Set up environment variables
os.environ["LANGCHAIN_API_KEY"] = "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"  # Replace with actual key
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "test1"  # Replace with your project name
os.environ["TOGETHER_API_KEY"] = "603944cea685d01bb254f1f1988f5629150a89e6c32856b0881191fc39344415"  # Replace with actual key


In [4]:
# Verify setup
verify_environment()
client = test_langsmith_connection()

# Create tracer with debug info
tracer = LangChainTracer(
    project_name=os.environ["LANGCHAIN_PROJECT"],
)
print(f"Tracer created for project: {os.environ['LANGCHAIN_PROJECT']} ✓")

callback_manager = CallbackManager([tracer])
print("Callback manager created ✓")



Environment variables verified ✓
LangSmith connection successful ✓
Tracer created for project: test1 ✓
Callback manager created ✓


In [5]:
# Initialize Together AI model
llm = ChatTogether(
model="meta-llama/Llama-3-70b-chat-hf",
temperature=0,
max_tokens=None,
timeout=None,
max_retries=2,
callbacks=[tracer],
# other params...
)

print("LLM initialized ✓")



LLM initialized ✓


In [6]:
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field, validator

In [7]:
class code(BaseModel):
    imports: str = Field(description="imports for the solution")
    code: str = Field(description="solution in python")


In [8]:
code_query = "write code for fibonacci series "

In [9]:
parser = PydanticOutputParser(pydantic_object=code)

prompt = PromptTemplate(
        template="""You are an expert programmer. Return ONLY a JSON object matching the following schema, with no additional text or explanation:

{format_instructions}

Query: {query}""",
        input_variables=["query"],
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )

chain = prompt | llm | parser

chain.invoke({"query": code_query})

code(imports='', code='def fibonacci(n):\n    fib_sequence = [0, 1]\n    while len(fib_sequence) < n:\n        fib_sequence.append(fib_sequence[-1] + fib_sequence[-2])\n    return fib_sequence\n\nprint(fibonacci(10))')

In [11]:
!pip install langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 3.6 MB/s eta 0:00:00


In [37]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from typing import Dict, TypedDict, Optional


In [101]:
class GraphState(TypedDict):
    question: Optional[str] = None
    generation: Optional[str] = None
    error: Optional[str] = None

In [102]:
workflow = StateGraph(GraphState)

In [103]:
import re
from langchain.prompts import PromptTemplate

def generate_code(state):
    query = state["question"]
    # print(query)
    parser = PydanticOutputParser(pydantic_object=code)

    if state.get("error") and state.get("generation"):
        template = """You are an expert programmer. Your previously generated code had errors.
I have mentioned the previously generated code and errors. Please regenerate the code.
Return ONLY a JSON object matching the following schema, with no additional text or explanation:

{format_instructions}

Query: {query}

Previously Generated Code:
{code}

Errors:
{error}"""

        # Define the prompt template
        prompt = PromptTemplate(
            template=template,
            input_variables=["query", "code", "error"],
            partial_variables={"format_instructions": parser.get_format_instructions()},
        )

        chain = prompt | llm | parser

        # Generate response with code and error
        generation = chain.invoke({
            "query": query,
            "code": state.get("generation", ""),
            "error": state.get("error", "")
        })

    else:


        # Define the prompt template
        prompt = PromptTemplate(
        template="""You are an expert programmer. Return ONLY a JSON object matching the following schema, with no additional text or explanation:

{format_instructions}

Query: {query}""",
        input_variables=["query"],
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )

        chain = prompt | llm | parser

        # Generate response without previous errors
        generation = chain.invoke({"query": query})

    return {"generation": generation}



In [104]:
def evaluate(state):
    namespace = {}
    generation = state.get("generation", {})

    # Convert Pydantic object to dictionary properly
    if hasattr(generation, "model_dump"):  # Pydantic v2+ support
        generation = generation.model_dump()

    errormessage = None  # Prevent UnboundLocalError

    try:
        # Execute imports
        imports_code = generation.get("imports", "")
        exec(imports_code, namespace)
        print("Imports executed successfully.")
    except Exception:
        errormessage = f"Error in imports execution:\n{traceback.format_exc()}"

    try:
        # Execute main code
        code = generation.get("code", "")
        exec(code, namespace)
        print("Main code executed successfully.")
    except Exception:
        errormessage = f"Error in main code execution:\n{traceback.format_exc()}"

    return {"error": errormessage} if errormessage else {}


In [105]:
workflow.add_node("generate_code", generate_code)
workflow.add_node("evaluate_code", evaluate)

In [106]:
workflow.add_edge("generate_code", "evaluate_code")

In [107]:
workflow.add_conditional_edges(
    "evaluate_code",
    lambda state: "generate_code" if state.get("error") else END  # Ensure the workflow terminates correctly
)

In [108]:
workflow.set_entry_point("generate_code")

In [109]:
app = workflow.compile()
inputs = {"question": "create 2 datastructures"}
result = app.invoke(inputs)
print(result)

Imports executed successfully.
Main code executed successfully.
{'question': 'create 2 datastructures', 'generation': code(imports='from collections import deque\nfrom typing import List', code='def create_datastructures() -> List:\n    datastructure1 = deque([1, 2, 3])\n    datastructure2 = [4, 5, 6]\n    return [datastructure1, datastructure2]')}
